In [ ]:
# coding:utf-8
import xlwt
import xlrd
from py2neo import Graph, Node, Relationship

##连接neo4j数据库，输入地址、用户名、密码
graph = Graph('http://localhost:7474', username='neo4j', password='root')

book = xlrd.open_workbook('桥梁结构及其病害整理.xlsx')
workSheetName = book.sheet_names() # ['桥梁结构', '桥梁病害', '桥梁结构关系', '病害关系', '桥梁和病害的关系']
print("Excel文件包含的表单有："+str(workSheetName))


# 根据指定的表单名,一行一行获取指定表单中的所有数据，表单名为worksheetname
def GetAllSheetCellValue(worksheetname):
    bridgeStructure = book.sheet_by_name(worksheetname)
    AllsheetValue = []
    for i in range(bridgeStructure.nrows):
        for j in range(bridgeStructure.ncols):
            AllsheetValue.append(bridgeStructure.cell_value(i,j))
    # print("《"+str(worksheetname)+"》"+"数据获取成功，请指定变量接收")
    return AllsheetValue

# 根据指定的表单名，按列获取表单中的数据
def GetAllSheetValueByColum(worksheetname):
    bridgeStructure = book.sheet_by_name(worksheetname)#获取指定名称的表单
    col_nums = bridgeStructure.ncols #获取指定表单的有效列数
    # print(bridgeStructure)
    AllsheetValue = []
    for i in range(col_nums):
        AllsheetValue.append(bridgeStructure.col_values(i))
    # print(AllsheetValue)
    return AllsheetValue

# 创建一个指定节点类，名字，创建节点,其他属性有需要自定拓展这个方法
def CreateNode(className,lableName,name):
    test_node= Node(className,lable = lableName, name=name)
    graph.create(test_node)

# 指定表名，类名，名字，批量创建节点
def CreateNodes(worksheetname,lableName,ClassName):
    sheetvalue = GetAllSheetCellValue(worksheetname)
    nums = 0
    for i in range(len(sheetvalue)):
        CreateNode(ClassName,lableName,sheetvalue[i])
        nums+=1
    print("创建"+worksheetname+"节点成功，总计创建%s个"%(nums))

# 根据需要创建节点的表名个数(有几个表就传输参数是几)，批量创建节点，这个方法中默认构件类名就是图谱中的类名
# 参数说明 nums：在一个Excel文件中的需要创建节点的表单数
def CreateNodesBySheetNums(nums):
    for i in range(nums):
        CreateNodes(workSheetName[i], workSheetName[i],workSheetName[i])

#建立两个节点之间的关系
#参数说明 node1：节点1，node2：节点2，relationship：节点之间的关系
def CreateTwoNodeRelationship(node1,node2,relationship):
    relation = Relationship(node2, relationship, node1)
    graph.create(relation)

# 根据Excel文件中两个对象列表和一个关系列表，建立两个列表之间的子类（subclassof）关系
# 参数说明 worksheetname：指定的表单名,className1:第一个类的名称，className2：第二个类的名称
def subclassRelationship(worksheetname, className1,className2):
    list1 = GetAllSheetValueByColum(worksheetname)[0]
    list2 = GetAllSheetValueByColum(worksheetname)[2]
    relationship = GetAllSheetCellValue(worksheetname)[1]
    print(list1,relationship,list2)
    # 利用Python执行CQL语句
    # graph.run("match(a:桥梁结构),(b:桥梁病害)  where a.name='支座' and b.name='冲蚀'  CREATE (a)-[r:DIRECTED]->(b)")

    num=0
    for i in range(len(list1)):
        num+=1
        graph.run("match(a:%s),(b:%s)  where a.name='%s' and b.name='%s'  CREATE (a)-[r:%s]->(b)"%(className1,className2,str(list2[i]),str(list1[i]),str(relationship)))
    print('创建%d个关系成功'%(num))

CreateNodesBySheetNums(2)
subclassRelationship(workSheetName[2],"桥梁结构","桥梁结构")
subclassRelationship(workSheetName[3],"桥梁病害","桥梁病害")
subclassRelationship(workSheetName[4],"桥梁病害","桥梁结构")

print("程序运行完成")



In [2]:
# -*- coding: utf-8 -*-
from py2neo import Graph, Node, Relationship, NodeMatcher
import pandas as pd
 
graph = Graph('http://localhost:7474', username='neo4j', password='243698cyc')
 
gjpath = r'C:\Users\13284\Desktop\cyc\1.xlsx'
gj = pd.read_excel(gjpath)       #如果是csv文件则将read_excel改为read_csv
gjlabel = list(gj)
country = Node("世界各国", 标签="世界各国")
for i in range(0, len(gj)):
    print(gj.loc[i][gjlabel[0]])
    gjnode = Node("国家", 标签=str(gj.loc[i][gjlabel[0]]))
    for l in gjlabel[1:]:
        print(gj.loc[i][l])
        gjnode[l] = str(gj.loc[i][l])
    graph.create(Relationship(country, "包含", gjnode))

国产半导体实力大阅兵
nan
nan
华天科技、UNISEMUnisem、通富微电、FABTRONIC SDN BHD
nan
nan
nan
nan
2020年第十五届“中国芯”优秀产品征集活动，专家评审会顺利召开
nan
杭州
nan
nan
nan
中国
nan
汽车成像的创新技术
nan
nan
nan
nan
汽车成像、激光雷达、毫米波雷达
nan
nan
复旦微科创板招股书中透露的FPGA实力
nan
nan
复旦微
nan
SerDes
nan
FPGA
一文读懂全球半导体格局
nan
nan
nan
nan
nan
nan
nan
晶圆厂产能不足下的思考
nan
nan
nan
nan
nan
nan
nan
中国大陆晶圆厂的“新挑战”
nan
nan
三星、台积电
nan
nan
nan
nan
如何维持全球半导体领先？美国专家给出四点建议
nan
nan
nan
nan
nan
nan
nan
外媒：中国半导体崛起的四大障碍
nan
nan
nan
nan
nan
nan
nan
Arm服务器芯片新战局
nan
欧洲
Arm
nan
nan
中国、美国、日本
nan
中芯国际确认美国出口管制属实，正在评估影响
nan
nan
中芯国际
nan
nan
nan
nan
1nm将如何实现？
nan
nan
nan
nan
FEOL、BEOL、MOL
nan
nan
芯谋专栏 | 解读涉嫌“出口管制”的中芯国际
nan
北美
中芯、华为、中兴、海思
美国商务部、美国商务部工业和安全局、SIA（美国半导体行业协会）、科创板、芯谋研究（ICwise)
nan
中国、美国
nan
FinFET的继任者：详解GAA晶体管
麦克迈克· 梅伯里博士
nan
英特尔、三星、台积电
IMEC、IBM
鳍式晶体管结构、鳍式结构、全环绕栅极技术、VLSI、离子注入（ion-implantation）、光刻、EUV、自对齐四重曝光、浅沟隔绝（STI Shallow Trench Isolation）
nan
nan
缺货涨价蔓延到硅片和封装领域
nan
nan
nan
nan
nan
nan
nan
LoRa不凉，反而在物联网时代大行其道！
nan
nan
nan
nan
LoRa
nan
nan
揭开英特尔制程命名的神秘面纱
nan
nan
nan
nan
SuperFi

功率器件散热新方案，冷却能力提升50倍
 Remco van Erp、 T.Brunschwiler、 Hansson J、] E.N.Wang、T.Acikalin、 T.-W.Wei、 D.B.Tuckerman、 R.J.Phillips、 G.M.Harpole、IEEE？Semi-Therm、 Badaroglu，M.
nan
nan
nan
nan
United States
nan
MIPS和Arm都无法幸免，为何受伤的都是IP巨头？
nan
nan
nan
nan
Arm 、MIPS
nan
nan
USB的前世今生
nan
nan
nan
nan
nan
nan
nan
外媒关注：华为力争多点突破“求生存”
郭平、周三
nan
华为、英特尔公司、高通公司、AMD
联合早报
5G
英国、美国、中国、澳大利亚、泰国、新加坡
nan
美国芯片IP实力大盘点
nan
nan
SST FPGA、eFPGA、MIPS、RISC-V NoC、Arteris
nan
nan
美国
nan
软件：给汽车中央网关“变个身”
丁大宇
nan
车联网
nan
服务型网关、中央网关
nan
nan
IC设计业的变与不变
nan
nan
nan
贝尔实验室
自动化、AI、拓扑、人类智能
nan
nan
从估值风险看中国集成电路产业的发展风险
nan
nan
碧桂园、恒大、百度、阿里、腾讯、红杉、高榕、高瓴、芯媒研究
nan
nan
中国
nan
?值得期待的十款国产芯片
nan
nan
nan
nan
FPGA、AI、5G
nan
nan
你不知道的苹果 A14 芯片的亮点
nan
nan
苹果公司
nan
5纳米制程技术
nan
nan
三星为高通生产全部5G芯片组，与台积电差距逐渐缩小
nan
nan
nan
nan
nan
中、美
nan
韩国公司正在低调布局AI领域
nan
nan
三星
nan
AI、人工智能
nan
nan
美国芯片制造水平究竟如何？能崛起吗？
Fuchs、Kirchain、Branstetter、Shalf、Gartner
中国台湾
BCG、IDM、OSAT
OECD
nan
美国、中国
nan
一寸市占一寸血，亚太半导体市场的成长太难了
nan
nan
德勤
nan
nan
中国
nan
台积电7nm再成焦点
nan
nan
nan
nan
n

刚刚！特朗普再签禁令！禁华为、海康威视...
特朗普
nan
华为、海康威视、浪潮集团、中国移动、中国电信、华为
养老基金、美国国防部
nan
nan
nan
联电将为英特尔代工芯片！
谢承儒
nan
英特尔、联电、台积电、三星、
nan
28纳米制程
nan
网通WiFi芯片
三星2年后赶上台积电：3nm独一无二！
 
nan
高通、NVIDIA、IBM、三星、台积电
nan
半导体晶圆代工、先进逻辑工艺、GAA环绕栅极晶体管
 
nan
刚刚！东芝回应出售晶圆厂！
  简山傑
大分市、岩手县
东芝、中国台湾联电、Japan Semiconductor 
8英寸晶圆厂、大分晶圆厂
电机控制和微处理器的模拟IC
中国台湾
nan
中芯国际的担当，“芯肝宝贝计划”实施第八次捐赠！
周子学、张文义、高永岗博士、周梅生、宋健、夏强
上海市
中芯国际
宋庆龄基金会、上海仁济医院
nan
nan
nan
富士康将在巴西、印度生产iPhone 12 Pro
nan
nan
苹果、富士康、
nan
5G、
巴西、中国、印度、日本、
nan
日本半导体厂火灾，烧了三天才灭！
nan
nan
旭化成、三星、东芝
旭化成微电子株式会社、
nan
日本、
nan
重磅！美国宣布禁运5nm、光刻技术！
nan
nan
ASML公司、资本国际集团(MSCI)、贝莱德集团、
nan
混合增材制造/计算机数控工具、特定的计算光刻软件有限的数字取证分析工具、亚轨道航天器
美国、荷兰、中国
5nm芯片
7nm芯片备货充足！
Abraham Liu、
nan
华为、谷歌、
nan
5G、
美国、中国、
麒麟芯片、
美国许可！台积电供货华为28nm芯片！
nan
nan
台积电、华为、Intel、AMD、中芯国际、
nan
nan
美国、
SoC芯片、麒麟710F芯片、海思芯片
重磅！中芯国际工艺获重大进展！
nan
nan
中芯国际、芯动科技有限公司、台积电、
nan
FinFET N+1先进工艺、
中国、
nan
重磅！海力士600亿收购！
nan
大连
SK海力士、英特尔、三星、
nan
英特尔NAND闪存及存储、傲腾、
nan
nan
总投资225亿!  14个半导体项目落地上海临港
朱芝松、
上海
nan
东方芯港、艾为消费电子芯片研发中心、
nan
中国、
nan
8寸晶圆代工产能紧张 交货

In [2]:
from py2neo import Graph, Node, Relationship, NodeMatcher
import pandas
import os
g = Graph('http://localhost:7474', username='neo4j', password='243698')
def importRelation():
    count=0
    #这里最好使用决定路径
    for root, dirs, files in os.walk(r'C:\Users\13284\Desktop\file'):
        for file in files:
            frame = pandas.read_excel(r"C:/Users/13284/Desktop/file/"+file)
            part = file.split('.')[0]
            for i in frame.index:
            #这里是我工作表的数据，也是我需要建立的节点
                name = frame.get_value(i, '标题')
                renming = frame.get_value(i,'人名')
                diming = frame.get_value(i, '地名')
                gongsiming = frame.get_value(i, '公司名')
              	#career_dict = {'部类': part}
                #part_node = Node('部类', name=part)
                #g.merge(part_node)
                 # 最后创建关系
                med_node = Node('新闻', name=name)  # label为节点标签，name为节点名称，需要注意不要用label='label'否则label会成为节点的的属性
                # '''如果把这个作为中药的属性，而不作为节点属性，不利于后面的问答系统'''
                alias_node=Node('人名',name=renming)
                smell_node=Node('地名',name=diming)
                cure_node=Node('公司名',name=gongsiming)
                # # med_node['别名'] = alias  # 向node添加属性'property'
                # # med_node['气味品质'] = smell
                # # med_node['使用方法'] = cure
                # # g.merge(med_node)  # 将节点加入图数据库与create不同之处在于若节点存在则不创
               # relat = Relationship(med_node, '属于', part_node)
                relat_one = Relationship(med_node, '人名是', alias_node)
                relat_two = Relationship(med_node, '地名是', smell_node)
                relat_three = Relationship(med_node, '公司是', cure_node)
                #g是我的neo4j账号，create可直接建立节点或者关系
                try:
                     g.create(relat_one)
                except:
                     continue
                try:
                     g.create(relat_two)
                except:
                     continue
                try:
                     g.create(relat_three)
                except:
                     continue
                try:
                     g.create(relat)
                except:
                     continue
                count+=1
                print(count)
importRelation()  

D:\anaconda\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
D:\anaconda\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app


KeyError: '人名'